In [1]:
import json

In [2]:
with open("00000.world.json") as f:
    d = json.load(f)

In [3]:
with open('00000.bidder.json') as f:
    b = json.load(f)

In [14]:
d['regionsMap']['adjacencyGraph'][0]['node']

{'id': 0, 'population': 514711, 'note': 'Newfoundland and Labrador'}

In [18]:
len(d['regionsMap']['adjacencyGraph'])

14

In [3]:
d.keys()

dict_keys(['regionsMap', 'bands', 'modelName', 'id', 'implementation'])

In [6]:
d['bands'][0].keys()

dict_keys(['baseCapacity', 'numberOfLots', 'licenses', 'synergies', 'uuid', 'name', 'worldId'])

In [12]:
d['bands'][0]['licenses'][21]['regionId']

7

In [7]:
d['bands'][0]['baseCapacity']

0.8937322357910741

In [25]:
len(d['bands'][2]['licenses'])

42

In [9]:
len(d['regionsMap']['adjacencyGraph'])

14

In [33]:
d['bands'][2]['synergies']

{'1': 1.1, '2': 1.2, '3': 1.1}

In [24]:
[x['name'] for x in d['bands']]

['UNPAIRED', 'LOW_PAIRED', 'HIGH_PAIRED']

In [15]:
import torch
from torch import nn

class ValueFunction(nn.Module):
    def __init__(self, items, device='cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__()
        self.items = items
        self.device = device

    def forward(self, bundle):
        raise NotImplementedError("This is abstract value function! :(")

In [138]:
class MRVMValueFunction(ValueFunction):
    def __init__(self, items, bidder, world, device='cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__(items, device)
        self.alpha = bidder['alpha']
        self.beta = list(bidder['beta'].values())
        self.R = len(world['regionsMap']['adjacencyGraph'])
        p = []
        for i in range(self.R):
            p.append(world['regionsMap']['adjacencyGraph'][i]['node']['population'])
        self.p = p
        self.zlow = list(bidder['zLow'].values())
        self.zhigh = list(bidder['zHigh'].values())
        self.t = bidder['setupType'].split()[3]
        self.bands = world['bands']
        self.bidder = bidder
        self.world = world

    def forward(self, x):
        output = 0
        for r in range(self.R):
            output += self.beta[r] * self.p[r] * self.sv(r, self.bandwidth(r, x.reshape(-1,))) * self.Gamma(r, x)
        return torch.tensor([output], device=self.device).float()

    def cap(self, b, r, x):
        c_b = self.bands[b]['baseCapacity']
        x_br = 0
        for i in range(self.bands[b]['numberOfLots']):
            if x[self.bands[b]['licenses'][r+i*self.R]['longId']] == 1:
                x_br += 1
        syn = 0
        if x_br > 0:
            syn = self.bands[b]['synergies'][str(x_br)]
        return c_b * x_br * syn
    
    def bandwidth(self, r, x):
        sum = 0
        for b in range(len(self.bands)):
            sum += self.cap(b, r, x)
        return sum
    
    def sv(self, r, c):
        p1 = 0
        p2 = self.zlow[r] * self.p[r] * self.beta[r]
        p3 = self.zhigh[r] * self.p[r] * self.beta[r]
        p4 = self.bandwidth(r, [1 for _ in range(len(self.items))])
        if (c >= p1) and (c <= p2):
            return ((c-p1)/(p2-p1)) * (0.27*self.alpha)
        if (c >= p2) and (c <= p3):
            return (((c-p2)/(p3-p2)) * (0.46*self.alpha)) + (0.27*self.alpha)
        return (((c-p3)/(p4-p3)) * (0.27*self.alpha)) + (0.73*self.alpha)
    
    def Gamma(self, r, x):
        if self.t == 'Local':
            if r in self.bidder['regionsOfInterest']:
                return 1
            else:
                return 0
        
            
        

In [139]:
items = list(range(98))
v = MRVMValueFunction(items, b, d)

In [142]:
x = [1 for _ in range(98)]
x[12] = 1
x[25] = 0
x[66] = 0
x = torch.tensor(x, device='cuda').float()
#x.reshape(1, -1)

In [127]:
x

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')

In [140]:
x = torch.ones(1, 98)

In [136]:
y = x.view(-1,)

In [134]:
x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.]])

In [143]:
v(x)

tensor([95624208.], device='cuda:0')

In [53]:
class DSFValueFunction(ValueFunction):
    def __init__(self, items, max_out, hidden_sizes, alpha, device='cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__(items, device)
        self.dsf =  DSF(len(items), 1, max_out, hidden_sizes, alpha).to(device)

    def forward(self, bundle):  # `bundle` can be a batch of bundles
        return self.dsf(bundle)

In [52]:
from comblearn.nn import DSF, Modular, Universe

In [64]:
i = list(range(10))
x = [1 for _ in range(10)]
x = torch.tensor(x, device='cuda').float()

In [65]:
vs = DSFValueFunction(i, 10, [2, 3, 2], 5)

In [70]:
vs(x).shape

torch.Size([1])